# **Stock Sentiment Analysis – Model Evaluation and Testing**

This notebook investigates various machine learning techniques and models for sentiment analysis of stock-related tweets, with the objective of identifying the most effective model for accurate sentiment prediction. Its companion notebook — see TK — applies the selected model to the problem context to generate final predictions.

## 0. Imports

In [ ]:
# Standard Library
import os
import re

# Local Scripts
os.chdir("../scripts")

# Constants
from constants import (
    TRAIN_DATA, 
    TEST_DATA, 
    LABELS, 
    DATA_DIR,
    NLTK_DATA,
    GENSIM_DATA
) #type: ignore

# Auxiliaries
from aux_funcs import (
    find_punctuated_tokens, 
    eval_sklearn_model, 
    eval_lstm_model,
    eval_llm_model, 
    eval_transformer
) #type: ignore

# Data Preprocessor
# from preprocessor import preprocess #type: ignore

# Local Classes
# os.chdir("../classes")
# from classes import W2VVectorizer, CLSVectorizer, BERTVectorizer

# Standard Data Visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Wordclound Visualization
from wordcloud import WordCloud

# Modelling and modelling metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Transformers
from transformers import AutoTokenizer, AutoModel

# Natural Language ToolKit
#import nltk
#from nltk.corpus import stopwords
#from nltk.stem import SnowballStemmer
#from nltk.stem.wordnet import WordNetLemmatizer

# General Similarity - NLP - Pre-trained Models
# from gensim.models import Word2Vec
# import gensim.downloader as api

/home/shadybea/anaconda3/envs/tm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-15 10:36:05.783534: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-15 10:36:05.797745: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749980165.814573   57874 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749980165.819410   57874 cuda_blas.cc:140

In [ ]:
# # Download if not data not found
# if not os.path.isdir(NLTK_DATA):
#     nltk.download('stopwords', download_dir=NLTK_DATA)
#     nltk.download('wordnet', download_dir=NLTK_DATA)

# nltk.data.path.append(NLTK_DATA)

## 1. Data Loading

In [ ]:
# train_data = pd.read_csv(TRAIN_DATA)
# test_data = pd.read_csv(TEST_DATA)

## 3. Data Preprocessing

In [ ]:
# Defining the stopwords
# stop = set(stopwords.words('english'))

# Initializing Lemmatizer
# lemma = WordNetLemmatizer()

# Initializing Stemmer
# stemmer = SnowballStemmer('english')

In [ ]:
# # Apply preprocessing
# train_data['text'] = preprocess(
#     corpus=train_data['text'],
#     stopwords=stop,
#     lemmatizer=lemma
# )

In [ ]:
# # Apply preprocessing
# test_data['text'] = preprocess(
#     corpus=test_data['text'],
#     stopwords=stop,
#     lemmatizer=lemma
# )

In [ ]:
# # Export preprocessed data
# train_data.to_csv(f'{DATA_DIR}/train_preprocessed.csv', sep=',', index=False)
# test_data.to_csv(f'{DATA_DIR}/test_preprocessed.csv', sep=',', index=False)

In [18]:
# Reassign datasets
train_data = pd.read_csv(f'{DATA_DIR}/train_preprocessed.csv')
test_data = pd.read_csv(f'{DATA_DIR}/test_preprocessed.csv')

In [ ]:
# Fill missing values with empty string
train_data['text'] = train_data['text'].fillna('')
test_data['text'] = test_data['text'].fillna('')

In [ ]:
# Drop rows with encoding errors
exceptions = find_punctuated_tokens(train_data['text'])
pattern = r'(?:' + '|'.join(re.escape(word) for word in exceptions) + r')'
train_data = train_data[~train_data['text'].str.contains(pattern, case=False, na=False)]

In [24]:
# Define dependent and independent features

# Train
X_train = train_data['text']
y_train = train_data['label']

# Test
X_test = test_data['text']

## 4. Feature Engineering

### 4.1 Bag of Words

In [26]:
# Bag of Words
bow = CountVectorizer(
    binary=True  # using binary=True, because working with short text
)

### 4.2 Word2Vec

In [27]:
w2v = W2VVectorizer(
    Word2Vec(
        sentences=train_data.text.apply(lambda x: x.split()).tolist(), #list of lists
        vector_size=5,    # size of the embedding vectors
        window=2,         # context window size
        min_count=1,      # minimum word frequency to include
        sg=1              # 1 for skip-gram; 0 for CBOW
    )
)

### 4.3. DistilBERT

In [28]:
distilbert_model = "distilbert-base-uncased-finetuned-sst-2-english"
distilbert = CLSVectorizer(
    embeddings_model=BERTVectorizer(
        model_name=distilbert_model
    ).embed
)

### 4.4. RoBERTa

In [29]:
roberta_model = "cardiffnlp/twitter-roberta-base-sentiment"
roberta = CLSVectorizer(
    embeddings_model=BERTVectorizer(
        model_name=roberta_model
    ).embed
)

Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 4.5. FinBERT

In [30]:
finbert_model = "yiyanghkust/finbert-tone"
finbert = CLSVectorizer(
    embeddings_model=BERTVectorizer(
        model_name=finbert_model
    ).embed
)

## 5. Modelling

In [32]:
# Stratified K-Fold
skf = StratifiedKFold(
    n_splits=5
    ,shuffle=True
    ,random_state=20
)

### 5.5. Language Model

In [ ]:
system_message = """
[REASON]
You are a financial sentiment analysis engine.

[TASK]
Analyze the input text ([INPUT]) from financial tweets and classify its sentiment as one of:
  0: Bearish (negative sentiment)
  1: Bullish (positive sentiment)
  2: Neutral (neutral sentiment)

Provide as an answer a single digit in [0, 1, 2]

[EXAMPLE:EXAMPLE OUTPUT]
  "JPMorgan reels in expectations on Beyond Meat": 0
  "Dougherty & Company starts at Buy": 1
  "Analysts React To FCC Decision On Intelsat C-Band Spectrum Auction": 2

[INPUT]
"""

In [ ]:
print("Now evaluating please hold...")

In [ ]:
# Evaluate the model
y_true_all, y_pred_all = eval_llm_model(
    model="tiiuae/falcon-7b-instruct"
    ,skf=skf
    ,X_train=X_train
    ,y_train=y_train
    ,system_message=system_message
)

In [ ]:
print("Overall classification report across all folds:\n")
print(classification_report(y_true_all, y_pred_all, digits=4, target_names=LABELS.keys()))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true_all, y_pred_all)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=LABELS.keys(), yticklabels=LABELS.keys())
plt.xlabel('Predicted', labelpad=15)
plt.ylabel('True', labelpad=15)
plt.title('Confusion Matrix (Language Model)', fontsize=16)
plt.show()